In [23]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.ascii 
from astropy import units as u

In [24]:
tab = astropy.io.ascii.read("circumstellardisks.csv")

In [25]:
print(tab.colnames)

['Object', 'Category', 'Spec Type', 'R band (mag)', 'Distance (pc)', 'Disk Major Axis &quot;', 'Disk Diameter (AU)', 'Inclination (degrees)', 'Resolution elements across', 'At ref. wavelength (microns)', 'RA (J2000)', 'DEC (J2000)']


## Get  targets (here use dec>0 cut)

In [26]:
from astropy.coordinates import Angle

k = 0
for i in np.arange(len(tab)):
    dec = Angle(tab["DEC (J2000)"][i] + " degree")
    if (dec.value > 0.0) & (tab["Inclination (degrees)"][i] < 60.0):
        k = k + 1
#         print(k, tab["Object"][i], tab["RA (J2000)"][i], tab["DEC (J2000)"][i], tab["R band (mag)"][i], tab["Distance (pc)"][i], \
#               tab["Inclination (degrees)"][i], tab["Category"][i])
        
        print("{0:2.0f} & {1:20s} & {2:20s} & {3:20s} & {4:6.1f} & {5:6.1f} \\\\".format(\
        k, tab["Object"][i], tab["RA (J2000)"][i], tab["DEC (J2000)"][i], tab["R band (mag)"][i], tab["Distance (pc)"][i], \
        ))
        

 1 & 99 Her               & 18 07 01.54          & +30 33 43.7          &    4.7 &   15.6 \\
 2 & AB Aur               & 04 55 45.93          & +30 33 03.6          &    7.1 &  144.0 \\
 3 & beta Leo             & 11 49 03.58          & +14 34 19.4          &    2.0 &   11.1 \\
 4 & beta Tri             & 02 09 32.63          & +34 59 14.3          &    2.9 &   38.9 \\
 5 & gamma Oph            & 17 47 53.56          & +02 42 26.2          &    3.7 &   29.1 \\
 6 & HD 104860            & 12 04 33.73          & +66 20 11.7          &    8.0 &   47.9 \\
 7 & HD 107146            & 12 19 06.50          & +16 32 53.9          &    6.7 &   27.5 \\
 8 & HD 15745             & 02 32 55.81          & +37 20 01.4          &    7.5 &   64.0 \\
 9 & HD 166               & 00 06 36.78          & +29 01 17.4          &    5.6 &   13.7 \\
10 & HD 183324            & 19 29 00.99          & +01 57 01.6          &    5.9 &   59.0 \\
11 & HD 74873             & 08 46 56.02          & +12 06 35.8        

![title](eq4.png)


In [27]:
# http://www.stsci.edu/~strolger/docs/UNITS.txt

import astropy.constants as ac

def cal_t0(mag=0.0, filt="K", SNR=5.0, R=1e5, pl_st_con=1e-5, aper=11.8, eta=0.1, q=0.95):
    if filt == "K":
        wav = 2.0 * u.micron
        f_zero = 3.90e-11 * u.erg / (u.cm)**2/ u.s / u.AA
    elif filt == "L":
        wav = 3.5 * u.micron
        f_zero = 8.1e-12 * u.erg / (u.cm)**2/ u.s / u.AA
        
    e_ph = ac.h * ac.c / wav
    f_zero = f_zero / e_ph
    
    f = f_zero / 2.512**(mag)
    A = np.pi * (aper / 2.0 * u.meter)**2 * 2.0 # two sub-apertures
    
    t0 = R * SNR**2 / (wav * pl_st_con**2 * f * A * q * eta)
    t0 = t0.decompose()
    
    return(t0)
    
    

![title](eq9.png)


In [28]:
def cal_tL(b=[1.0], w=[1e-1], gamma=[2], t0 = None, eta_p=1e-1):
    tL = 0.0 * u.s
    for bi, wi, gi in zip(b, w, gamma):
        tL = tL + t0 / (eta_p)**2 * (bi * wi)**gi
    return(tL)

![](eq12.png)


In [29]:
def cal_tbg(t0=None, eta_p=1e-1, mag_bg=0.0, mag=0.0, fiber_area=1.0):
    # https://www2.keck.hawaii.edu/inst/nirc2/filters.html
    # skg bg = 12.2 in K and 2.9 in L
    f_bg_over_f_star = 2.512**(mag - mag_bg) * fiber_area # fiber_area in arcsec**2
    tbg = f_bg_over_f_star * t0 / (eta_p)**2
    return(tbg)

![](eq13.png)


In [30]:
def cal_dc(mag=0.0, filt="K", R=1e5, aper=11.8, eta=0.1, q=0.95, t0=None, eta_p=1e-1, i_dark=1.0, pixel_sampling=3.0):
    if filt == "K":
        wav = 2.0 * u.micron
        f_zero = 3.90e-11 * u.erg / (u.cm)**2/ u.s / u.AA
    elif filt == "L":
        wav = 3.5 * u.micron
        f_zero = 8.1e-12 * u.erg / (u.cm)**2/ u.s / u.AA
        
    e_ph = ac.h * ac.c / wav
    f_zero = f_zero / e_ph
    
    f = f_zero / 2.512**(mag)
    A = np.pi * (aper / 2.0 * u.meter)**2
    
    signal_pixel = (f * wav *  A * q * eta / (R * pixel_sampling)).decompose()
    dc_pixel = i_dark / u.s
    
    tdc = (dc_pixel / signal_pixel * t0 / (eta_p)**2).decompose()
    
    return(tdc)

![](eq14.png)

In [31]:
def cal_trd(Nrd=2.0, eta_s=1e-5, eta_p=1e-1, t0=None, W=2**16):
    trd = Nrd**2 / W * eta_s / (eta_p)**2 * t0
    return(trd)


![](eq7.png)

In [32]:
def cal_tfs(Rs=1.0, dis=10.0, aper=22.8, filt="K", t0=None, eta_p=1e-1):
    if filt == "K":
        wav = 2.0 * u.micron
    elif filt == "L":
        wav = 3.5 * u.micron

    Rs = Rs * u.solRad
    dis = dis * u.parsec
    aper = aper * u.meter
    
    tfs = (Rs / dis / np.sqrt(12) * aper / wav)**2 / (eta_p)**2 * t0
    tfs = tfs.decompose()
    return(tfs)

## Use Mamajek table to get K and L band magnitude

In [33]:
tab_mmj = astropy.io.ascii.read("mamajek_table.csv")

In [34]:
print(tab_mmj.colnames)

['SpT', 'Teff', 'logT', 'logL', 'Mbol', 'BCv', 'Mv', 'B-V', 'Bt-Vt', 'G-V', 'Bp-Rp', 'G-Rp', 'M_G', 'b-y', 'U-B', 'V-Rc', 'V-Ic', 'V-Ks', 'J-H', 'H-Ks', 'Ks-W1', 'W1-W2', 'W1-W3', 'W1-W4', 'M_J', 'M_Ks', 'i-z', 'z-Y', 'R_Rsun', 'Msun', '#SpT']


In [35]:
def cal_mag(mag_r, filt="K", spt="A1"):
    for i in np.arange(len(tab_mmj)):
        if tab_mmj["SpT"][i][0:2] == spt:
            V_Rc = tab_mmj["V-Rc"][i]
            V_Ks = tab_mmj["V-Ks"][i]
            Ks_W1 = tab_mmj["Ks-W1"][i]
            V_Rc, V_Ks, Ks_W1 = (float(V_Rc), float(V_Ks), float(Ks_W1))
            break
    if filt == "K":
        return(mag_r + (V_Rc - V_Ks))
    elif filt == "L":
        return(mag_r + (V_Rc - V_Ks) - Ks_W1)
    else:
        return(999)
    

## Now let's put all togher to calculate the required exposure time for a target

In [36]:
b_arr = np.array([2.00, 0.43, 0.50, 0.31, 0.21, 0.39, 0.06, 0.04, 0.17, 0.09])
w_arr = np.array([1e-2] * len(b_arr))
gamma = np.array([2] * len(b_arr))

def calc_total_time(star_name=None, pl_st_con=1e-5, eta_p=1e-1, filt="K", mag_bg=12.2, \
                    b_arr=b_arr, w_arr=w_arr, gamma=gamma, eta=0.1, boost_factor=1600):
    
    eta_s = np.sum((b_arr * w_arr)**gamma)

    ind = np.where(tab["Object"] == star_name)[0][0]
    aper = 8.4
    baseline = 22.4
    q = 0.95
    R = 1e5
    pixel_sampling = 3.0
    SNR = 5.0
    Rs = 1.0
    
    dis = tab["Distance (pc)"][ind]
    
    mag_r = tab["R band (mag)"][ind]
    mag = cal_mag(mag_r, filt=filt, spt=tab["Spec Type"][ind][0:2])
    
    if filt == "K":
        wav = 2.0 * u.micron
    elif filt == "L":
        wav = 3.5 * u.micron
    lambda_D = ((wav / (baseline * u.meter)).decompose() * u.radian).to(u.arcsec)
        
    if baseline != aper:
        fiber_area = np.pi * ((((1.4 * wav / (baseline * u.meter) / 2.0).decompose()) * u.radian).to(u.arcsec).value)**2
        fiber_area = fiber_area * (baseline / aper / np.sqrt(2))
    else:
        fiber_area = np.pi * ((((1.4 * wav / (baseline * u.meter) / 2.0).decompose()) * u.radian).to(u.arcsec).value)**2

    Nrd = 2.0
    W = 2**16
    i_dark = 1e-2

    t0 = cal_t0(mag=mag, filt=filt, SNR=SNR, R=R, pl_st_con=pl_st_con, aper=aper, eta=eta, q=q)

    tfs = cal_tfs(Rs=Rs, dis=dis, aper=baseline, filt=filt, t0=t0, eta_p=eta_p)

    tL = cal_tL(b=b_arr, w=w_arr, gamma=gamma, t0 = t0, eta_p=eta_p)

    tbg = cal_tbg(t0=t0, eta_p=eta_p, mag_bg=mag_bg, mag=mag, fiber_area=fiber_area)

    trd = cal_trd(Nrd=Nrd, eta_s=eta_s, eta_p=eta_p, t0=t0, W=W)

    tdc = cal_dc(mag=mag, filt=filt, R=R, aper=aper, eta=eta, q=q, t0=t0, eta_p=eta_p, i_dark=i_dark, \
             pixel_sampling=pixel_sampling)

    t_total = tfs + tL + tbg + trd + tdc

    with open(star_name+".txt", "wb") as f:
        f.write(bytes("\multicolumn{4}{c}{\\bf{Star Parameters}}\\\\\n", "UTF-8"))
        f.write(bytes("star name\t\t&=& {0:<30s}&\\\\\n".format(star_name), "UTF-8"))
        f.write(bytes("magnitude\t\t&=& {0:<6.2f} &mag\\\\\n".format(mag), "UTF-8"))
        f.write(bytes("distance\t\t&=& {0:<10.2f} &pc\\\\\n".format(dis), "UTF-8"))
        f.write(bytes("star radius\t\t&=& {0:<6.2f} &solar radii\\\\\n".format(Rs), "UTF-8"))
        f.write(bytes("planet-star contrast\t&=& {0:<8.2e}&\\\\\n".format(pl_st_con), "UTF-8"))
        f.write(bytes("\multicolumn{4}{c}{\\bf{Telescope Parameters}}\\\\\n", "UTF-8"))
        f.write(bytes("aperture\t\t&=& {0:<8.2f} &meter\\\\\n".format(aper), "UTF-8"))
        f.write(bytes("baseline\t\t&=& {0:<8.2f} &meter\\\\\n".format(baseline), "UTF-8"))
        f.write(bytes("wavelength\t\t&=& {0:<8.2f} &um\\\\\n".format(wav.value), "UTF-8"))
        f.write(bytes("lambda/D\t\t&=& {0:<8.2f} &mas\\\\\n".format(lambda_D.value * 1e3), "UTF-8"))
        f.write(bytes("\multicolumn{4}{c}{\\bf{Coronagraph Parameters}}\\\\\n", "UTF-8"))
        f.write(bytes("filter\t\t\t&=& {0:<10s}&\\\\\n".format(filt), "UTF-8"))
        f.write(bytes("starlight suppression\t&=& {0:<8.2e}&\\\\\n".format(eta_s), "UTF-8"))
        f.write(bytes("planet throughput\t&=& {0:<8.2e}&\\\\\n".format(eta_p), "UTF-8"))
        f.write(bytes("bmn array\t\t&=& {0:<100s}&\\\\\n".format(str(b_arr)), "UTF-8"))
        f.write(bytes("aberration array\t&=& {0:<100s}&\\\\\n".format(str(w_arr)), "UTF-8"))
        f.write(bytes("gamma array\t&=& {0:<100s}&\\\\\n".format(str(gamma)), "UTF-8"))
        f.write(bytes("\multicolumn{4}{c}{\\bf{Spectrograph Parameters}}\\\\\n", "UTF-8"))
        f.write(bytes("required SNR\t\t&=& {0:<8.1f}&\\\\\n".format(SNR), "UTF-8"))
        f.write(bytes("background\t\t&=& {0:<6.2f} &mag / arcsec**2\\\\\n".format(mag_bg), "UTF-8"))
        f.write(bytes("spectral resolution\t&=& {0:<8.0f}&\\\\\n".format(R), "UTF-8"))
        f.write(bytes("pixel sampling rate\t&=& {0:<5.1f} &pixel per resolution element\\\\\n".format(pixel_sampling), "UTF-8"))
        f.write(bytes("system throughput\t&=& {0:<6.3f}&\\\\\n".format(eta), "UTF-8"))
        f.write(bytes("fiber size\t\t&=& {0:<8.2e} &arcsec**2\\\\\n".format(fiber_area), "UTF-8"))
        f.write(bytes("\multicolumn{4}{c}{\\bf{Detector Parameters}}\\\\\n", "UTF-8"))
        f.write(bytes("dark current\t\t&=& {0:<8.2e} &electron/pixel/s\\\\\n".format(i_dark), "UTF-8"))
        f.write(bytes("readout noise\t\t&=& {0:<8.4f} &electron/read\\\\\n".format(Nrd), "UTF-8"))
        f.write(bytes("quantum efficiency\t&=& {0:<6.3f}&\\\\\n".format(q), "UTF-8"))
        f.write(bytes("well depth\t\t&=& {0:<8.0f} &ADU\\\\\n".format(W), "UTF-8"))
        f.write(bytes("\multicolumn{4}{c}{\\bf{Exposure Times}}\\\\\n", "UTF-8"))
        f.write(bytes("t0\t\t\t&=& {0:<10.5e} &s\\\\\n".format(t0.value), "UTF-8"))
        f.write(bytes("finite star time\t&=& {0:<10.5e} &s\\\\\n".format(tfs.value), "UTF-8"))
        f.write(bytes("low order time\t\t&=& {0:<10.5e} &s\\\\\n".format(tL.value), "UTF-8"))
        f.write(bytes("background time\t\t&=& {0:<10.5e} &s\\\\\n".format(tbg.value), "UTF-8"))
        f.write(bytes("readout noise time\t&=& {0:<10.5e} &s\\\\\n".format(trd.value), "UTF-8"))
        f.write(bytes("dark current time\t&=& {0:<10.5e} &s\\\\\n".format(tdc.value), "UTF-8"))
        f.write(bytes("total time\t\t&=& {0:<10.5e} &s\\\\\n".format(t_total.value), "UTF-8"))
        f.write(bytes("total time\t\t&=& {0:<10.2f} &hour\\\\\n".format(t_total.to(u.hour).value), "UTF-8"))
        f.write(bytes("boost factor\t\t&=& {0:<8.1f} &\\\\\n".format(boost_factor), "UTF-8"))
        f.write(bytes("total time\t\t&=& {0:<10.2f} &hour\\\\\n".format(t_total.to(u.hour).value/boost_factor), "UTF-8"))

    print("total time = ", t_total.to(u.hour))
    print("More detals in "+star_name+".txt")
    print("{0:20s} & {1:6.1f} & {2:5.1f} & {3:6.1f} & {4:10.2f} & \\\\\n".format(\
        star_name, dis, mag, lambda_D.value * 1e3, t_total.to(u.hour).value/boost_factor))
    

In [37]:
b_arr = np.array([2.09, 0.48, 2.26, 0.26, 2.41, 0.36, 2.38, 0.03, 2.36, 0.06])
w_arr = np.array([1e-2] * len(b_arr))
gamma = np.array([2,2,4,2,4,2,4,2,4,2])
calc_total_time(star_name="HD 104860", pl_st_con=1e-6, eta_p=2e-1, filt="K", mag_bg=12.2, \
                b_arr=b_arr, w_arr=w_arr, eta=0.1, gamma=gamma, boost_factor=1600.0)

total time =  71963.20857311727 h
More detals in HD 104860.txt
HD 104860            &   47.9 &   7.0 &   18.4 &      44.98 & \\



In [38]:
calc_total_time(star_name="99 Her", pl_st_con=1e-6, eta_p=2e-1, filt="L", mag_bg=2.2, \
                b_arr=b_arr, w_arr=w_arr, eta=0.1, gamma=gamma, boost_factor=1225.0)

total time =  128934.62770257556 h
More detals in 99 Her.txt
99 Her               &   15.6 &   3.7 &   32.2 &     105.25 & \\



In [39]:
calc_total_time(star_name="AB Aur", pl_st_con=1e-6, eta_p=2e-1, filt="K", mag_bg=12.2, \
                b_arr=b_arr, w_arr=w_arr, eta=0.1, gamma=gamma, boost_factor=1600.0)

total time =  75426.06750237073 h
More detals in AB Aur.txt
AB Aur               &  144.0 &   7.1 &   18.4 &      47.14 & \\

